In [5]:
#Ładuję potrzebne biblioteki
import pandas as pd
import numpy as np
from tkinter import filedialog
from tkinter import *
import os
import glob

In [6]:
# Tworzymy listę i dwa puste DF
listOfFiles = []
DFL = pd.DataFrame()
DFM = pd.DataFrame() 

In [7]:
# Funkcja pobiera listę ścieżek
def get_filenames():
    Tk().withdraw()
    print("Initializing Dialogue... \nPlease select a file.")
    tk_filenames = filedialog.askdirectory()
    tempDir = tk_filenames
    return tempDir


choosen_dir = get_filenames()
# glob zbiera listę plików 
os.chdir(choosen_dir)
for file in glob.glob("*.csv"):
    listOfFiles.append(file)

# Do wcześniej stworzonych DF dodajemy kolejno pliki w zależności od tego na jaką literę plik się zaczyna.
for file in listOfFiles:
    if file.startswith('L'):
        Table = pd.read_csv(file, sep=',')
        DF1 = pd.DataFrame(Table)
        DFL = DFL.append(DF1, sort=False)
    elif file.startswith('M'):
        Table2 = pd.read_csv(file, sep=',')
        DF2 = pd.DataFrame(Table2)
        DFM = DFM.append(DF2, sort=False)

Initializing Dialogue... 
Please select a file.


In [8]:
# Deklaruje listę kolumn którę będą mi potrzebne
DFL['NMIOT'] = 1
cols1 = ['LPROS1', 'SUTKI_L', 'SUTKI_P', 'LRASA1', 'LDUR', 'LDOP', 'LRASAOJ', 'LIL11', 'LIL21','LWAGA1','M_WAGUR','LDBRAK','NMIOT','LREJ1',
        'LWOJ1', 'LSEK1', 'LSTA1','NROJM1','LSUT1', 'NRMAM1', 'NOJMIO']
DFL = DFL[cols1]

In [9]:
# Zmieniam nazwy kolumn tak by odpowiadały kolumnon z DFL
DFM['LSUT1'] = np.nan  
DFM = DFM.rename(columns={'LDUR': 'LDUR1'})
DFM = DFM.rename(columns={'LDOP': 'LDUR'})
DFM = DFM.rename(columns={'LDUR1': 'LDOP'})
DFM = DFM.rename(columns={'RASAOJ': 'LRASAOJ'})
DFM = DFM.rename(columns={'NRO': 'NROJM1'})
DFM = DFM.rename(columns={'LREJ': 'LREJ1'})
DFM = DFM.rename(columns={'LWOJ': 'LWOJ1'})
# Deklaruje listę kolumn którę będą mi potrzebne
cols2 = ['LPROS1', 'LRASA1', 'LDUR', 'LDOP', 'LRASAOJ', 'LIL11', 'LIL21','LWAGA1','M_WAGUR','LDBRAK','NMIOT','LREJ1',
        'LWOJ1', 'LSEK1', 'LSTA1','LSUT1','NROJM1']
DFM = DFM[cols2]

In [10]:
# Wypełniam NaN'y zerami
DFL['SUTKI_L'].fillna(0, inplace=True)
DFL['SUTKI_P'].fillna(0, inplace=True)
DFL['LSUT1'].fillna(0, inplace=True)
# Rozdzielam sutki prawe od lewych
DFL['PSUT'] = DFL['LSUT1'].astype(str).str[1]
DFL['LSUT1'] = DFL['LSUT1'].astype(str).str[0]
DFL['PSUT'].replace('.','0',inplace = True)

In [11]:
# Convertuje liczby sutków do inta
convert_dict = {'LSUT1': int,
                'PSUT': int
               }
  
DFL = DFL.astype(convert_dict)

In [12]:
# Liczę sumę sutków prawych i lewych
DFL['ISUT'] = DFL['PSUT'] + DFL['LSUT1'] + DFL['SUTKI_P'] + DFL['SUTKI_L']

In [13]:
# Wprowadzamy przedział czasowy z jakiego będą pobieranie dane
date1 = input('Date range from YYYY-MM-DD') 
date2 = input('Date range to YYYY-MM-DD')

Date range from YYYY-MM-DD 2020-12-12
Date range to YYYY-MM-DD 2021-12-11


In [14]:
# Funkcja zamienia format kolumn na format daty
class ToDate():

    def change_to_date(self, a, b):
        a[b] = pd.to_datetime(a[b])


dat = ToDate()
dat.change_to_date(DFL, 'LDOP')
dat.change_to_date(DFM, 'LDOP')
dat.change_to_date(DFL, 'LDUR')
dat.change_to_date(DFM, 'LDUR')

In [15]:
# W DF zostawaimy tylko dane z wcześniej zadeklarowanego przedziału czasowego
DFL = DFL[(DFL['LDOP'] >= date1) & (DFL['LDOP'] <= date2)]
DFM = DFM[(DFM['LDOP'] >= date1) & (DFM['LDOP'] <= date2)]

In [16]:
# Wyliczamy:
# Datę pierwszego oprosienia
DFL['POP'] = DFL['LDOP'] - DFL['LDUR']
# Okres międzymiotu
DFM['OMM'] = DFM['LDOP'] - DFM['LDUR']
# Zamieniam ww wartości na dni
DFL['POP'] = DFL['POP'] / np.timedelta64(1, 'D')
DFM['OMM'] = DFM['OMM'] / np.timedelta64(1, 'D')

In [17]:
# W zmiennej DF3 łączymy poprzednie DF
DF3 = DFL.append(DFM)

In [18]:
DF3 = DF3.rename(columns={'LDUR': 'DURL'})
DF3 = DF3.rename(columns={'LDOP': 'DURM'})

In [19]:
# Wyliczamy ile prosiąt nie dożyło 21 dnia życia
DF3['STRATY'] = DF3['LIL11'] - DF3['LIL21']
# Wyliczamy wartość % strat
DF3['ST21'] = (DF3['STRATY'] / DF3['LIL11']) * 100

In [20]:
# Funkcja tworzy DF z określonymi już kolumnami
def createDT(name):
    temp = pd.DataFrame(columns = ['Rasa','liczba loch','liczba ocenionych miotów ogółem', 
                                   'liczba ocenionych miotów pierw.','Liczba prosiąt urodzonych w miocie',
                                   'Liczba prosiąt w 21 dniu życia','Straty prosiąt do 21 dnia życia','Liczba sutków lochy',
                                   'Wiek w dniu pierwszego oprosienia(dni)','Okres miedzymiotu(dni)'],index =[1])
    temp['Rasa'] = name
    return temp

# Funkcja sortuje dane po odpowiedniej rasie
def sortedDT(DT, col, number):
    newDT = DT[DT[col] == number]
    return newDT
# Funkcja sortuje dane po rasie matki i rasie ojca
def sorted2DT(DT,number1,number2):
    newDT = DT[(DT.LRASA1 == number1) & (DT.LRASAOJ == number2)]
    return newDT

def months(DT, col, col2):
    newDT = DT[DT[col] != DT[col2]]
    return newDT

# Funkcja wylicza wszystkie niezbędne wartości i przypisuje je do DF
def all_count(DF3, new):
    new['liczba loch'] = len(DF3.drop_duplicates(subset='LPROS1', keep="first").copy())
    new['liczba ocenionych miotów ogółem'] = len(DF3)
    new['liczba ocenionych miotów pierw.'] = len(DF3[DF3['NMIOT']==1])
    new['Liczba prosiąt urodzonych w miocie'] = np.around(DF3['LIL11'].mean(), decimals=2)
    new['Liczba prosiąt w 21 dniu życia'] = np.around(DF3['LIL21'].mean(), decimals=2)
    new['Straty prosiąt do 21 dnia życia'] = np.around(DF3['ST21'].mean(), decimals=2)
    new['Wiek w dniu pierwszego oprosienia(dni)'] = np.around(DF3['POP'].mean())
    new['Okres miedzymiotu(dni)'] = np.around(DF3['OMM'].mean())
    new['Liczba sutków lochy'] = np.around(DF3['ISUT'].mean(), decimals=2)

    return new

In [21]:
WBPxWBP = createDT('WBP x WBP')
wbpXwbp = sorted2DT(DF3,10,10)
all_count(wbpXwbp,WBPxWBP)

WBPxPBZ = createDT('WBP x PBZ')
wbpXpbz = sorted2DT(DF3,10,20).copy()
all_count(wbpXpbz,WBPxPBZ)

PBZxPBZ = createDT('PBZ x PBZ')
pbzXpbz = sorted2DT(DF3,20,20).copy()
all_count(pbzXpbz,PBZxPBZ)

PBZxWBP = createDT('PBZ x WBP')
pbzXwbp = sorted2DT(DF3,20,10).copy()
all_count(pbzXwbp,PBZxWBP)

DUROCxDUROC = createDT('Duroc x Duroc')
durocXduroc = sorted2DT(DF3,70,70).copy()
all_count(durocXduroc,DUROCxDUROC)

DUROCxPIETRAIN = createDT('Duroc x Pietrain')
durocXpietrain = sorted2DT(DF3,70,80).copy()
all_count(durocXpietrain,DUROCxPIETRAIN)

PIETRAINxPIETRAIN = createDT('Pietrain x Pietrain')
pietrainXpietrain = sorted2DT(DF3,80,80).copy()
all_count(pietrainXpietrain,PIETRAINxPIETRAIN)

PIETRAINxDUROC = createDT('Pietrain X Duroc')
pietrainXduroc = sorted2DT(DF3,80,70).copy()
all_count(pietrainXduroc,PIETRAINxDUROC)

,Rasa,liczba loch,liczba ocenionych miotów ogółem,liczba ocenionych miotów pierw.,Liczba prosiąt urodzonych w miocie,Liczba prosiąt w 21 dniu życia,Straty prosiąt do 21 dnia życia,Liczba sutków lochy,Wiek w dniu pierwszego oprosienia(dni),Okres miedzymiotu(dni)
1,Pietrain X Duroc,204,286,55,11.53,10.95,4.81,14.22,390.0,178.0


In [22]:
krzyzowkiDF = pd.concat([WBPxWBP,WBPxPBZ,PBZxPBZ,PBZxWBP,DUROCxDUROC,DUROCxPIETRAIN,PIETRAINxPIETRAIN,PIETRAINxDUROC])

In [23]:
krzyzowkiDF.set_index('Rasa',inplace = True)

In [24]:
krzyzowkiDF

,liczba loch,liczba ocenionych miotów ogółem,liczba ocenionych miotów pierw.,Liczba prosiąt urodzonych w miocie,Liczba prosiąt w 21 dniu życia,Straty prosiąt do 21 dnia życia,Liczba sutków lochy,Wiek w dniu pierwszego oprosienia(dni),Okres miedzymiotu(dni)
Rasa,,,,,,,,,
WBP x WBP,927,1355,265,12.60,11.66,6.97,15.17,379.0,169.0
WBP x PBZ,881,1323,274,12.60,11.80,5.76,15.08,355.0,165.0
PBZ x PBZ,2096,3354,650,12.61,11.60,7.46,15.35,359.0,173.0
PBZ x WBP,662,943,198,12.84,12.16,5.06,15.45,359.0,170.0
Duroc x Duroc,129,162,39,12.04,11.10,7.31,13.41,366.0,176.0
Duroc x Pietrain,244,343,76,11.68,10.74,7.47,13.30,374.0,176.0
Pietrain x Pietrain,91,111,14,11.77,10.71,8.68,13.50,412.0,175.0
Pietrain X Duroc,204,286,55,11.53,10.95,4.81,14.22,390.0,178.0
